## Jupyter Notebook for extracting raw data from Syglass-generated labels to generate pseudocolored independent images for visualization

import os
import numpy as np
from tifffile import imread, imwrite
import pandas as pd
from skimage.measure import regionprops_table

In [2]:
image_location = r'D:\Raphael\Aivia\11-52-32_AC300_Whole_PCW6-5_4X_06Z_5-MHC_6-chAT_7-Pax7_4u_lr_Sy16b_ROI-57_Image'
mask_location = r'D:\Raphael\Aivia\11-52-32_AC300_Whole_PCW6-5_4X_06Z_5-MHC_6-chAT_7-Pax7_4u_lr_Sy16b_ROI-57_Mask'
result_folder = r'D:\Raphael\Aivia\masked_c'

In [3]:
# Iterate each image in the folder
for image in os.listdir(image_location):
    print("getting statistics on: "+image)
    
    img = imread(image_location+"/"+image)
    short_name = os.path.splitext(image)
    
    # Iterate through the masks (labels) generated in Syglass
    for labels in os.listdir(mask_location):
        lbls = imread(mask_location+"/"+labels)
        import time
        start_time = time.time()

        # get dictionary of measurements 
        props = regionprops_table(lbls)

        print("Determining label statistics using scikit-image took " + str(time.time() - start_time) + " s")  
                          
        # read out arrays of values
        content = pd.DataFrame(props)
        

        label_vector = content['label']
        content.to_csv(result_folder+'/'+str(short_name[0])+'.csv')

content
            
        

getting statistics on: 11-52-32_AC300_Whole_PCW6-5_4X_06Z_5-MHC_6-chAT_7-Pax7_4u_lr_Sy16b_ROI-57_C-1_Z_aivia.aivia.tif
Determining label statistics using scikit-image took 2.4863057136535645 s


,label,bbox-0,bbox-1,bbox-2,bbox-3,bbox-4,bbox-5
0,1,239,0,388,425,773,768
1,2,66,26,589,438,435,1161
2,3,259,0,255,358,493,728


In [7]:
c=1
a=0

# Extract the raw pixels for each mask and generate an independent file for each.

for index in range(0, label_vector.max()):
    start_time = time.time()
    output_data = np.empty_like(img)
    output_data = np.where(lbls==c, img, 0)
    output_data = output_data.astype(img.dtype)
    name= '/'+str(short_name[0])+'_label_C'+str(a)+'.tif'
    imwrite(result_folder+'/'+name, output_data, compression ='zlib')
    print("Creating mask "+str(c)+" tooks "+ str(time.time() - start_time) + " s")  
    c+=1
    a+=1
    continue
     
    

Creating mask 1 tooks 6.9958226680755615 s
Creating mask 2 tooks 6.578093767166138 s
Creating mask 3 tooks 6.704489707946777 s
